In [4]:
import torch
import torch.nn as nn
import torch.optim as op 



In [2]:
import pickle

# Load the dataset
with open("train_loader1.pkl", "rb") as f:
    train_loader = pickle.load(f)

print('Train loader loaded')

with open("test_loader1.pkl", "rb") as g:
    test_loader = pickle.load(g)

print('Test loader loaded')


AttributeError: Can't get attribute 'Dataset' on <module '__main__'>

In [11]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# Define the custom Dataset class (must match the original definition)
class Dataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

        print("Initial DataFrame size:", len(self.df))

        # Filter out rows where the image does not exist
        self.df = self.df[self.df.iloc[:, 0].apply(lambda x: os.path.exists(os.path.join(img_dir, x)))].reset_index(drop=True)

        print("Final DataFrame size:", len(self.df))

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_name = self.df.iloc[index, 0]
        img_path = os.path.join(self.img_dir, img_name)

        image = Image.open(img_path)

        label = self.df.iloc[index, -1]
        label = torch.tensor(label, dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

# Load the train_loader
train_loader = torch.load("train_loader2.pth", weights_only=False)
print('Train loader loaded')

# Load the test_loader (if applicable)
test_loader = torch.load("test_loader2.pth", weights_only=False)
print('Test loader loaded')

AttributeError: Can't get attribute 'custom_collate_fn' on <module '__main__'>

In [ ]:
class CRNN(nn.Module):
    def __init__(self, num_classes, input_channels=1):
        super(CRNN, self).__init__()

        # CNN Feature Extractor
        self.cnn = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # Only height pooling

            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # Only height pooling

            nn.Conv2d(512, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU()
        )

        # Recurrent Layers (LSTM)
        self.rnn = nn.LSTM(512, 256, bidirectional=True, batch_first=True, num_layers=2)

        # Fully Connected Layer for Character Prediction
        self.fc = nn.Linear(256 * 2, num_classes)  # 2x for bidirectional

    def forward(self, x):
        # CNN Feature Extraction
        x = self.cnn(x)
        x = x.squeeze(2)  # Remove the height dimension (H=1 after pooling)
        x = x.permute(0, 2, 1)  # (Batch, Width, Channels) → (Batch, Seq_len, Features)

        # Pass through LSTM
        x, _ = self.rnn(x)

        # Fully connected layer
        x = self.fc(x)

        return x


In [ ]:
ctc_loss = nn.CTCLoss(blank=0)  # Define CTC Loss (blank index = 0)

def train(model, dataloader, optimizer, epochs=10, device="cuda"):
    model.train()
    model.to(device)
    
    for epoch in range(epochs):
        total_loss = 0

        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)  # (Batch, Seq_len, Num_classes)
            log_probs = F.log_softmax(outputs, dim=2)

            # Compute CTC Loss
            input_lengths = torch.full((log_probs.size(1),), log_probs.size(0), dtype=torch.long)
            label_lengths = torch.tensor(label_lengths, dtype=torch.long)
            loss = ctc_loss(log_probs.permute(1, 0, 2), targets, input_lengths, label_lengths)

            # Backpropagation
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

